In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re, pickle, os

## Customize file names

In [2]:
staff_str = 'ufpb-professor-staff'

## Scrape UFPB's professor staff list 

- Data retrieval occurs: 
    - either by web scraping Department's websites
    - or by loading pre-serialized data (if any)

- Use `scrape_on=True` for updated web scraping

In [3]:
# read list of departments and their associated links
dep_list = pd.read_csv('../input/lista-departamentos-THE.csv')

# if on, retrieve data from UFPB portal
scrape_on = False

# if on, export sheets to CSV
to_csv = True

In [4]:
staff_pkl = os.path.join(os.pardir,'pkl',staff_str + '.pkl')

if scrape_on:

    data = {}

    for dep in range(len(dep_list)):

        # department name
        dep_name = dep_list['nome_departamento'][dep]

        # request department's professors
        html = urlopen(dep_list['url_corpoDocente'][dep])
        bs = BeautifulSoup(html.read(),'html.parser')

        # get professor descriptions from HTML tree
        staff = bs.body.find_all('td',{"class": "descricao"})

        # open list per department
        prof_list = []

        # loop over staff members
        for member in staff:    

            # prepare to store
            prof = {}

            # get full name and degree
            for m in member.find_all('span',{'class':'nome'}):        
                name_str = m.getText().strip()
                fullname = name_str.split('\n')[0]            

                # missing degree
                rex = re.search('(\(\w+\))',name_str) 
                if rex:
                    degree = rex.group()[1:-1]
                else:
                    degree = 'missing'                

                # store
                prof['fullname'] = fullname[:-2] # remove a last '\t'
                prof['degree'] = degree

            # get SIGGA public page (extract Lattes from here!!!)
            for m in member.find_all('span',{'class':'pagina'}):
                for link in m.find_all('a',href=True):

                    # store by adding absolute URL
                    page = 'https://sigaa.ufpb.br' + link['href']
                    prof['sigaa-page'] = page  

            # store professor
            prof_list.append(prof)

            data[dep_name] = prof_list

    # pickle data          
    with open(staff_pkl,'wb') as f:
        pickle.dump(data,file=f)
    
else:
    # load data
    with open(staff_pkl,'rb') as f:
        data = pickle.load(f,encoding='utf8')
    

## Report per Department (no international classification)

- In this section, we produce a simple report per department comprising:
    - total number of professors
    - number of masters
    - number of doctors
- Also, we compute these totals for the entire university and export them to .XLSX files for easy handling from ready readability by non-technical analysts.

In [5]:
staff_xlsx = os.path.join(os.pardir,'xlsx','no-class','staff')

total_profs = []
total_msc = []
total_dsc = []

# go through departments
for k in data.keys():
    
    # get members
    staff = data[k]
    name, degree = [],[]
    for member in staff:
        name.append(member['fullname'])
        degree.append(member['degree'])
    df = pd.DataFrame({'nome':name,'titulo':degree}).sort_values(by='nome')    
    
    # compute stats
    total_members = len(df)
        
    tk = df['titulo'].value_counts().keys().to_list()    
    tv = df['titulo'].value_counts().values
    
    if tk.count('MESTRE') == 1 and tk.count('DOUTOR') == 1:
        msc_members, dsc_members = tv[0], tv[1]
        msc_ratio = msc_members/total_members
        dsc_ratio = dsc_members/total_members
    elif tk.count('MESTRE') == 0 and tk.count('DOUTOR') == 1:
        msc_members, dsc_members = 0, tv[0]
        msc_ratio = msc_members/total_members
        dsc_ratio = dsc_members/total_members
    elif tk.count('MESTRE') == 1 and tk.count('DOUTOR') == 0:
        msc_members, dsc_members = tv[0], 0
        msc_ratio = msc_members/total_members
        dsc_ratio = dsc_members/total_members
            
    stats = pd.DataFrame({'n_membros':[total_members],
                            'n_msc':[msc_members],
                            'n_dsc':[dsc_members],
                            'razao_msc':[msc_ratio],
                            'razao_dsc':[dsc_ratio]})  
    
    # store to total
    total_profs.append(total_members)
    total_msc.append(msc_members)
    total_dsc.append(dsc_members)
    
    file_save = os.path.join(staff_xlsx, k + '.xlsx')
    with pd.ExcelWriter(file_save) as writer:
        df.to_excel(writer,sheet_name='members',index=False,encoding='utf-8')
        stats.to_excel(writer,sheet_name='stats',index=False,encoding='utf-8')  
        

# save totals        
total_profs = sum(total_profs)
total_msc = sum(total_msc)
total_dsc = sum(total_dsc)

totals = pd.DataFrame({'n_docentes':[total_profs],
                            'n_msc':[total_msc],
                            'n_dsc':[total_dsc]})  

# EXCEL
file_save = os.path.join(staff_xlsx,'TOTAL.xlsx')
with pd.ExcelWriter(file_save) as writer:
    totals.to_excel(writer,sheet_name='professors',index=False,encoding='utf-8')

## Report per Department (THE Ranking classification)

- _Remark:_ while running this code on macOS with M1 processor, I found an inconsistency of encoding in this part of the code: 
`area = dep_list[dep_list['nome_departamento'] == n]['THE_classificacao'].values[0]`.

- An _IndexError_ was returned because no matching was found for this particular string 'DEPARTAMENTO DE ENGENHARIA DE PRODUÇÃO'. While cheking character by character, somehow I found different Unicodes for the last 3 characters (35th - 37th), as seen below. 

```
35 Ç != C
36 Ã != ̧
37 O != A
```

**=> Not sure if this has something to do with processor, but this problem was not occurring in Intel. Anyway, I removed the latin characters from the input list.**

In [6]:
staff_xlsx_the = os.path.join(os.pardir,'xlsx','the','staff')

# go through departments by reading pre-existing files
segments = []
for f in os.listdir(staff_xlsx):
    if not f.startswith('.') and not f == 'TOTAL.xlsx':
        n,_ = f.split('.')
        df = pd.read_excel(os.path.join(staff_xlsx,f),sheet_name='stats')       
        area = dep_list[dep_list['nome_departamento'] == n]['THE_classificacao'].values[0]
        members = df['n_membros'].values[0]
        segments.append((area,members))   
        
# get THE classes        
the_classes = set(dep_list['THE_classificacao'])

# sums members per THE class
totals_the = {}
for c in the_classes:    
    aux = []
    for k,v in segments:
        if k == c:
            aux.append(v)
    totals_the[c] = sum(aux)
    
areas = []
ns = []
for k,v in totals_the.items():    
    areas.append(k)
    ns.append(v)  
    
file_save = os.path.join(staff_xlsx_the, 'TOTAL.xlsx')    
df_the = pd.DataFrame({'areas':areas,'docentes':ns}).sort_values(by='areas')
df_the.to_excel(file_save,index=False,encoding='utf-8')

## CSV conversion

- Reports are converted to .CSV in batch mode and exported to CSV directory at once.  

In [7]:
if to_csv:

    staff_csv = os.path.join(os.pardir,'csv')

    for f in os.listdir(staff_xlsx):
        if not f.startswith('.') and not f == 'TOTAL.xlsx':
            n,_ = f.split('.')
            save = os.path.join(staff_csv,n + ' - MEMBERS.csv')
            temp = pd.read_excel(os.path.join(staff_xlsx,f),sheet_name='members')               
            temp.to_csv(save,index=False)
            save = os.path.join(staff_csv,n + ' - STATS.csv')
            temp = pd.read_excel(os.path.join(staff_xlsx,f),sheet_name='stats')               
            temp.to_csv(save,index=False)

        elif f == 'TOTAL.xlsx':  
            n,_ = f.split('.')
            temp = pd.read_excel(os.path.join(staff_xlsx,f))
            save = os.path.join(staff_csv,n + '.csv')
            temp.to_csv(save,index=False)        